## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.83,77,100,20.04,light rain
1,1,Atuona,PF,-9.8000,-139.0333,78.31,73,21,17.69,few clouds
2,2,Vanimo,PG,-2.6741,141.3028,77.45,88,100,7.18,heavy intensity rain
3,3,Pevek,RU,69.7008,170.3133,-3.44,95,100,2.15,overcast clouds
4,4,Sumbawanga,TZ,-7.9667,31.6167,70.52,67,56,4.63,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.83,77,100,20.04,light rain
1,1,Atuona,PF,-9.8000,-139.0333,78.31,73,21,17.69,few clouds
2,2,Vanimo,PG,-2.6741,141.3028,77.45,88,100,7.18,heavy intensity rain
4,4,Sumbawanga,TZ,-7.9667,31.6167,70.52,67,56,4.63,light rain
5,5,Rocha,UY,-34.4833,-54.3333,83.34,34,5,15.21,clear sky
6,6,Castro,BR,-24.7911,-50.0119,69.17,64,95,6.26,overcast clouds
11,11,Mahebourg,MU,-20.4081,57.7000,80.26,94,75,3.44,light rain
15,15,Puerto Ayora,EC,-0.7393,-90.3518,75.16,76,43,7.49,scattered clouds
16,16,Namibe,AO,-15.1961,12.1522,75.24,62,81,12.28,broken clouds
17,17,Arraial Do Cabo,BR,-22.9661,-42.0278,73.36,94,100,5.75,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                314
City                   314
Country                310
Lat                    314
Lng                    314
Max Temp               314
Humidity               314
Cloudiness             314
Wind Speed             314
Current Description    314
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.83,77,100,20.04,light rain
1,1,Atuona,PF,-9.8000,-139.0333,78.31,73,21,17.69,few clouds
2,2,Vanimo,PG,-2.6741,141.3028,77.45,88,100,7.18,heavy intensity rain
4,4,Sumbawanga,TZ,-7.9667,31.6167,70.52,67,56,4.63,light rain
5,5,Rocha,UY,-34.4833,-54.3333,83.34,34,5,15.21,clear sky
...,...,...,...,...,...,...,...,...,...,...
666,666,Saint-Joseph,RE,-21.3667,55.6167,80.96,83,28,8.01,light rain
667,667,Mabaruma,GY,8.2000,-59.7833,82.81,59,13,7.36,few clouds
671,671,Acapulco,MX,16.8634,-99.8901,75.02,83,75,4.61,broken clouds
672,672,Malappuram,IN,11.0667,76.0667,80.76,74,20,2.30,haze


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.83,light rain,-23.1203,-134.9692,
1,Atuona,PF,78.31,few clouds,-9.8000,-139.0333,
2,Vanimo,PG,77.45,heavy intensity rain,-2.6741,141.3028,
4,Sumbawanga,TZ,70.52,light rain,-7.9667,31.6167,
5,Rocha,UY,83.34,clear sky,-34.4833,-54.3333,
6,Castro,BR,69.17,overcast clouds,-24.7911,-50.0119,
11,Mahebourg,MU,80.26,light rain,-20.4081,57.7000,
15,Puerto Ayora,EC,75.16,scattered clouds,-0.7393,-90.3518,
16,Namibe,AO,75.24,broken clouds,-15.1961,12.1522,
17,Arraial Do Cabo,BR,73.36,moderate rain,-22.9661,-42.0278,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] =f"{lat}, {lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))